In [ ]:
!pip install konlpy

!pip install transformers==3.0.2
!pip install tor|ch==2.0.0
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
!pip install kobert-transformers

In [ ]:
from konlpy.tag import Komoran
komoran = Komoran()

from transformers import AutoTokenizer, AutoModelForMaskedLM
tokenizer = AutoTokenizer.from_pretrained("monologg/kobert")
model = AutoModelForMaskedLM.from_pretrained("monologg/kobert")

import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

Some weights of BertForMaskedLM were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


###문장 압축 모델

In [ ]:
sentence = input()

'수상한 파트너'가 한순간도 눈을 뗄 수 없는 전개 속 충격 반전을 펼치며 안방극장을 쑥대밭으로 만들었다


In [ ]:
import re

def compress_sentence(sentence):

  # 문장 어절 단위로 토큰화
  def word_tokenizer(sentence):
    return sentence.split(' ')

  token = word_tokenizer(sentence)

  # 따옴표와 따옴표를 포함하는 문장을 하나의 토큰으로
  def process_quoted_words(tokens):
    processed_tokens = []
    quoted_word = ""
    in_quote = False

    for token in tokens:
      if "'" in token:
        if in_quote:
          quoted_word += " " + token
          processed_tokens.append(quoted_word)
          quoted_word = ""
          in_quote = False
        else :
          quoted_word = token
          in_quote = True
      else :
        if in_quote:
          quoted_word += " " + token
        else:
          processed_tokens.append(token)
    return processed_tokens
  token = process_quoted_words(token)

  # 형태소 분석 후 언어정보점수 추가
  def add_linguistic_score(sentence):
    pos = komoran.pos(sentence)
    score = 0
    for p in pos:
      if p[1] == 'NNP': # 고유명사
        score += 0.0001
      elif p[1] == 'NNG': # 일반명사
        score += 0.0001
      elif p[1] == 'vv': # 동사 -> 서술어
        score += 0.03
      elif p[1] == 'SL' : # 외국어
        score += 0.0003
      elif p[1] == 'JKS' : # 주격 조사 -> 주어
        score += 0.01
      elif p[1] == 'JKO' : # 목적격 조사 -> 목적어
        score += 0.001
    quoted_words = re.findall(r"'(.*?)'", sentence)
    for quoted_word in quoted_words: # 따옴표 안의 단어들
      score += 0.000001
    return score

  # 압축 문장 후보군 생성 및 언어정보점수 계산
  # n-gram
  compressed_candidates = []
  max_n = len(token) - 4 # 문장의 주성분인 주어,목적어,보어,서술어는 최소한 남겨두기 위해
  for n in range(1, max_n + 1):
    for i in range(len(token) - n + 1):
      compressed_tokens = token[:i] + token[i+n:]
      compressed_sentence = " ".join(compressed_tokens)
      # 언어정보점수 계산
      score = add_linguistic_score(compressed_sentence)
      # 압축 문장 후보에 언어정보점수 추가
      compressed_candidates.append((compressed_sentence,score,n))

  # perplexity 계산
  def calculate_perplexity_score(sentence):

    # KoBERT 모델이 이해할 수 있는 형태로 tokenize
    tokens = tokenizer.tokenize(sentence)
    token_ids = tokenizer.convert_tokens_to_ids(tokens) # tokenized word -> 정수 인덱스

    # [MASK] 토큰 위치 찾기
    mask_token_index = token_ids.index(tokenizer.mask_token_id)

    # 입려값과 출력값 생성
    input_ids = torch.tensor([token_ids])
    outputs = model(input_ids) # 예측값
    predictions = outputs[0]

    # [MASK] 토큰 위치에 대한 예측값 추출
    masked_predictions = predictions[0, mask_token_index]

    # softmax 함수를 이용하여 확률값을 확률 분포로 변환
    probs = torch.softmax(masked_predictions, dim = -1)

    # perplexity 계산
    perplexity = torch.exp(torch.mean(torch.log(probs)))

    return perplexity.item()

  perplexity_scores = []
  compressed_candidates_with_score = []

  for n in range(1,max_n+1):
    for i in range(len(token) - n + 1 ):
      mask_idx = list(range(i, i+n))
      masked_tokens = list(token)
      for j in mask_idx:
        masked_tokens[j] = "[MASK]"
      masked_sentence = " " .join(masked_tokens)

      # perplexity score 계산
      perplexity_score = calculate_perplexity_score(masked_sentence)

      # 언어정보점수 가져오기
      linguistic_score = compressed_candidates[i][1]

      # 최종 점수 = perplexity_score + 언어정보점수
      final_score = perplexity_score * linguistic_score

      # perplexity_score 및 언어정보점수 저장
      perplexity_scores.append(perplexity_score)
      compressed_candidates_with_score.append((re.sub(r'\[MASK\]\s*', '', masked_sentence), final_score, n))

      # 결과 출력
      print("압축 문장 후보 : {}".format(' '.join(masked_tokens)))
      print(f"perplexity score : {perplexity_score}. 언어정보점수 : {linguistic_score}")
      print(f"최종 점수 : {final_score}\n")

  # 최종 압축 문장 선택
  compressed_candidates_with_score_sorted = sorted(compressed_candidates_with_score, key=lambda x: x[1])

  # 정렬된 리스트에서 '압축 문장 후보'와 '최종 점수'
  result = [(re.sub(r'\[MASK\]\s*', "'", candidate[0]), candidate[1], candidate[2]) for candidate in compressed_candidates_with_score_sorted]

  df = pd.DataFrame(result, columns=['압축 문장 후보', '최종 점수', 'n-gram'])
  df['n-gram'] = df['n-gram'].astype(int)
  pd.options.display.float_format = '{:.15f}'.format

  display(df)

  # 최종 점수가 가장 낮은 압축 문장 선택
  final_compressed_sentence = re.sub(r'\[MASK\]\s*', '', compressed_candidates_with_score_sorted[0][0])
  selected_n = compressed_candidates_with_score_sorted[0][2]

  print(f"n-gram: {selected_n}")
  print(f"입력 문장: {sentence}")
  print(f"최종 압축 문장: {final_compressed_sentence}")

  return final_compressed_sentence

In [ ]:
compress_sentence(sentence)

압축 문장 후보 : [MASK] 한순간도 눈을 뗄 수 없는 전개 속 충격 반전을 펼치며 안방극장을 쑥대밭으로 만들었다
perplexity score : 3.292207111371681e-05. 언어정보점수 : 0.0038
최종 점수 : 1.2510387023212388e-07

압축 문장 후보 : '수상한 파트너'가 [MASK] 눈을 뗄 수 없는 전개 속 충격 반전을 펼치며 안방극장을 쑥대밭으로 만들었다
perplexity score : 3.4587690606713295e-05. 언어정보점수 : 0.013800999999999996
최종 점수 : 4.7734471806325e-07

압축 문장 후보 : '수상한 파트너'가 한순간도 [MASK] 뗄 수 없는 전개 속 충격 반전을 펼치며 안방극장을 쑥대밭으로 만들었다
perplexity score : 3.615337482187897e-05. 언어정보점수 : 0.012800999999999995
최종 점수 : 4.627993510948725e-07

압축 문장 후보 : '수상한 파트너'가 한순간도 눈을 [MASK] 수 없는 전개 속 충격 반전을 펼치며 안방극장을 쑥대밭으로 만들었다
perplexity score : 3.845615719910711e-05. 언어정보점수 : 0.013900999999999995
최종 점수 : 5.345790412247877e-07

압축 문장 후보 : '수상한 파트너'가 한순간도 눈을 뗄 [MASK] 없는 전개 속 충격 반전을 펼치며 안방극장을 쑥대밭으로 만들었다
perplexity score : 3.545775689417496e-05. 언어정보점수 : 0.013900999999999995
최종 점수 : 4.92898278585926e-07

압축 문장 후보 : '수상한 파트너'가 한순간도 눈을 뗄 수 [MASK] 전개 속 충격 반전을 펼치며 안방극장을 쑥대밭으로 만들었다
perplexity score : 3.202321022399701e-05. 언어정보점수 : 0.01390099999

,압축 문장 후보,최종 점수,n-gram
0,눈을 뗄 수 없는 전개 속 충격 반전을 펼치며 안방극장을 쑥대밭으로 만들었다,0.000000119731242,2
1,한순간도 눈을 뗄 수 없는 전개 속 충격 반전을 펼치며 안방극장을 쑥대밭으로 만들었다,0.000000125103870,1
2,없는 전개 속 충격 반전을 펼치며 안방극장을 쑥대밭으로 만들었다,0.000000131315082,5
3,수 없는 전개 속 충격 반전을 펼치며 안방극장을 쑥대밭으로 만들었다,0.000000132080590,4
4,전개 속 충격 반전을 펼치며 안방극장을 쑥대밭으로 만들었다,0.000000132137532,6
...,...,...,...
90,'수상한 파트너'가 한순간도 수 없는 전개 속 충격 반전을 펼치며 안방극장을 쑥대밭...,0.000000492058996,2
91,'수상한 파트너'가 한순간도 눈을 뗄 없는 전개 속 충격 반전을 펼치며 안방극장을 ...,0.000000492898279,1
92,'수상한 파트너'가 한순간도 눈을 뗄 수 없는 충격 반전을 펼치며 안방극장을 쑥대밭...,0.000000505835656,2
93,'수상한 파트너'가 한순간도 눈을 없는 전개 속 충격 반전을 펼치며 안방극장을 쑥대...,0.000000507243554,2


n-gram: 2
입력 문장: '수상한 파트너'가 한순간도 눈을 뗄 수 없는 전개 속 충격 반전을 펼치며 안방극장을 쑥대밭으로 만들었다
최종 압축 문장: 눈을 뗄 수 없는 전개 속 충격 반전을 펼치며 안방극장을 쑥대밭으로 만들었다


'눈을 뗄 수 없는 전개 속 충격 반전을 펼치며 안방극장을 쑥대밭으로 만들었다'

###모델 성능 평가

####모델성능평가용 함수

In [ ]:
import re

def compress_sentence_test(sentence): # 모델 성능 평가용 compress_sentence 함수

  # 교정된 문장 어절 단위로 토큰화
  def word_tokenizer(sentence):
    return sentence.split(' ')

  token = word_tokenizer(sentence)

  # 따옴표와 따옴표를 포함하는 문장을 하나의 토큰으로
  def process_quoted_words(tokens):
    processed_tokens = []
    quoted_word = ""
    in_quote = False

    for token in tokens:
      if "'" in token:
        if in_quote:
          quoted_word += " " + token
          processed_tokens.append(quoted_word)
          quoted_word = ""
          in_quote = False
        else :
          quoted_word = token
          in_quote = True
      else :
        if in_quote:
          quoted_word += " " + token
        else:
          processed_tokens.append(token)
    return processed_tokens

  token = process_quoted_words(token)

  # 형태소 분석 후 언어정보점수 추가
  def add_linguisitic_score(sentence):
    pos = komoran.pos(sentence)
    score = 0
    for p in pos:
      if p[1] == 'NNP' : # 고유명사
        score += 0.00001 # 임의로
      elif p[1] == 'NNG' : # 일반명사
        score += 0.0001 # 임의로
      elif p[1] == 'vv' : # 동사 -> 서술어
        score += 0.0000001 # 임의로
      elif p[1] == 'SL' : # 외국어
        score += 0.0003 # 임의로
      elif p[1] == 'JKS' : # 주격 조사 -> 주어
        score += 0.003
      elif p[1] == 'JKO' : # 목적격 조사 -> 목적어
        score += 0.003
    quoted_words = re.findall(r"'(.*?)'", sentence)
    for quoted_word in quoted_words:
        score += 0.000001
    return score

  # 압축 문장 후보군 생성 및 언어정보점수 계산
  # n-gram
  compressed_candidates = []
  token_length = len(token)
  max_n = 2 if token_length <= 4 else token_length - 4
  for n in range(1, max_n + 1):
    for i in range(token_length - n + 1):
      compressed_tokens = token[:i] + token[i+n:]
      compressed_sentence = " ".join(compressed_tokens)
      # 언어정보점수 계산
      score = add_linguisitic_score(compressed_sentence)
      # 압축 문장 후보에 언어정보점수 추가
      compressed_candidates.append((compressed_sentence, score, n))

  def calculate_perplexity_score(sentence):

    # KoBERT 모델이 이해할 수 있는 형태로 tokenize
    tokens = tokenizer.tokenize(sentence)
    token_ids = tokenizer.convert_tokens_to_ids(tokens) # tokenized word -> 정수 인덱스

    # [MASK] 토큰 위치 찾기
    mask_token_index = token_ids.index(tokenizer.mask_token_id)

    # 입력값과 출력값 생성
    input_ids = torch.tensor([token_ids])
    outputs = model(input_ids) # 예측값
    predictions = outputs[0]

    # [MASK] 토큰 위치에 대한 예측값 추출
    masked_predictions = predictions[0, mask_token_index]

    # softmax 함수를 이용하여 확률값을 확률 분포로 변환
    probs = torch.softmax(masked_predictions, dim=-1)

    # perplexity 계산
    perplexity = torch.exp(torch.mean(torch.log(probs)))

    return perplexity.item()

  perplexity_scores = []
  compressed_candidates_with_score = []

  for n in range(1,max_n + 1):
    for i in range(len(token) - n + 1):
      mask_idx = list(range(i, i + n))
      masked_tokens = list(token)
      for j in mask_idx:
        masked_tokens[j] = "[MASK]"
      masked_sentence = " ".join(masked_tokens)

      # perplexity score 계산
      perplexity_score = calculate_perplexity_score(masked_sentence)

      # 언어정보점수 가져오기
      score = compressed_candidates[i][1]

      # perplexity_score + 언어정보점수
      final_score = perplexity_score * score

      # perplexity_score 및 언어정보점수 저장
      perplexity_scores.append(perplexity_score)
      compressed_candidates_with_score.append((re.sub(r'\[MASK\]\s*', '', masked_sentence), final_score, n))

  # 최종 압축 문장 선택
  compressed_candidates_with_score_sorted = sorted(compressed_candidates_with_score, key=lambda x: x[1])
  # 정렬된 리스트에서 '압축 문장 후보'와 '최종 점수'
  result = [(re.sub(r'\[MASK\]\s*', "'", candidate[0]), candidate[1], candidate[2]) for candidate in compressed_candidates_with_score_sorted]

  df = pd.DataFrame(result, columns=['압축 문장 후보', '최종 점수', 'n-gram'])
  df['n-gram'] = df['n-gram'].astype(int)
  pd.options.display.float_format = '{:.15f}'.format


  # 최종 점수가 가장 낮은 압축 문장 선택
  final_compressed_sentence = re.sub(r'\[MASK\]\s*', '', compressed_candidates_with_score_sorted[0][0])
  selected_n = compressed_candidates_with_score_sorted[0][2]

  return final_compressed_sentence

####모델 성능 평가

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def calculate_similarity(compressed_sentence, reference_sentence):

    # 형태소 분석 및 형태소 단위로 문장 분리
    compressed_morphemes = komoran.morphs(compressed_sentence)
    reference_morphemes = komoran.morphs(reference_sentence)

    # 모든 형태소 종류 추출
    all_morphemes = list(set(compressed_morphemes + reference_morphemes))

    # 형태소 빈도수 계산
    compressed_freq = {}
    reference_freq = {}

    for morpheme in all_morphemes:
        compressed_freq[morpheme] = compressed_morphemes.count(morpheme)
        reference_freq[morpheme] = reference_morphemes.count(morpheme)

    # 형태소 빈도수를 벡터로 변환
    compressed_vector = list(compressed_freq.values())
    reference_vector = list(reference_freq.values())

    # 코사인 유사도 계산
    similarity = cosine_similarity([compressed_vector], [reference_vector])[0][0]
    return similarity

# 파일 경로 설정
source_file = '/content/drive/MyDrive/23-1/소프트웨어캡스톤디자인/Data/source_축약 전 원문 파일.txt'
target_file = '/content/drive/MyDrive/23-1/소프트웨어캡스톤디자인/Data/target_정답 축약 문장 파일.txt'

import random

# 문장 압축 결과와 정답 문장을 저장할 리스트
compressed_sentences = []
reference_sentences = []

# 'source_축약 전 원문 파일.txt'에서 문장 읽어오기
with open(source_file, 'r', encoding='utf-8') as f:
    compressed_sentences = f.readlines()

# 'target_정답 축약 문장 파일.txt'에서 문장 읽어오기
with open(target_file, 'r', encoding='utf-8') as f:
    reference_sentences = f.readlines()

# 문장 개수
num_sentences = len(compressed_sentences)

# 각 시행의 평균을 저장할 리스트
averages = []

# 10번 시행
n = 50 # n개의 문장 인덱스 선택
for _ in range(10):
    # 랜덤하게 n개의 문장 인덱스 선택
    random_indices = random.sample(range(num_sentences), n)

    # 전체 유사도와 선택된 문장 개수 초기화
    total_similarity = 0
    selected_sentences = 0

    # 랜덤하게 선택된 문장들에 대해 유사도 계산
    for i in random_indices:
        try:
            compressed_sentence = compress_sentence_test(compressed_sentences[i].strip())
            reference_sentence = reference_sentences[i].strip()

            # 유사도 계산
            similarity_score = calculate_similarity(compressed_sentence, reference_sentence)
            total_similarity += similarity_score
            selected_sentences += 1

        except IndexError:
            print(f"문장 {i+1} - 오류 발생: 리스트 인덱스 범위 초과")
            print("오류를 건너뛰고 다음 문장으로 넘어갑니다.")
            print()
            continue

    # 평균 계산
    average_similarity = total_similarity / selected_sentences
    averages.append(average_similarity)

    # 결과 출력
    print(f"시행 {_+1}")
    print(f"선택된 문장 개수: {selected_sentences}")
    print(f"전체 유사도 평균: {average_similarity}")
    print()

# 최종 평균 계산
final_average = sum(averages) / len(averages)
print(f"10번의 최종 평균: {final_average}")

문장 2393 - 오류 발생: 리스트 인덱스 범위 초과
오류를 건너뛰고 다음 문장으로 넘어갑니다.

문장 74 - 오류 발생: 리스트 인덱스 범위 초과
오류를 건너뛰고 다음 문장으로 넘어갑니다.

시행 1
선택된 문장 개수: 48
전체 유사도 평균: 0.5674722839971902

문장 68 - 오류 발생: 리스트 인덱스 범위 초과
오류를 건너뛰고 다음 문장으로 넘어갑니다.

시행 2
선택된 문장 개수: 49
전체 유사도 평균: 0.45979321867280565

문장 495 - 오류 발생: 리스트 인덱스 범위 초과
오류를 건너뛰고 다음 문장으로 넘어갑니다.

문장 2423 - 오류 발생: 리스트 인덱스 범위 초과
오류를 건너뛰고 다음 문장으로 넘어갑니다.

시행 3
선택된 문장 개수: 48
전체 유사도 평균: 0.4876786876239439

문장 1577 - 오류 발생: 리스트 인덱스 범위 초과
오류를 건너뛰고 다음 문장으로 넘어갑니다.

시행 4
선택된 문장 개수: 49
전체 유사도 평균: 0.5652313231645194

문장 2809 - 오류 발생: 리스트 인덱스 범위 초과
오류를 건너뛰고 다음 문장으로 넘어갑니다.

문장 2024 - 오류 발생: 리스트 인덱스 범위 초과
오류를 건너뛰고 다음 문장으로 넘어갑니다.

시행 5
선택된 문장 개수: 48
전체 유사도 평균: 0.5628553606371552

